# Replication Factor Calculator v1.1
### Patrick Perrine
This is to help compute solutions of the term $r$, or *replication factor*, as specified by:

Valiant, L.G. (2005). Memorization and Association on a Realistic Neural Model. *Neural Computation, 17*, 527–555. https://doi.org/10.1162/0899766053019890

A couple known solutions of $r$ from Valiant (2005) are given in the example checks following each set of conditions listed. This notebook has not been fully validated by checking all known solutions from the cited paper and related work as of yet.

All condition checks should be as stringent than the conditions checked when searching for the solutions given in the cited article, except for Equation 2.1 and its variants as described below. 

### Notes on Precision for Equations 2.1, 2.1', and 2.1''

We will designate the *supposed* solution of $r$ when passed to the check functions as $r_{s}$, and refer to the *expected* value of $r$ from the calculations in the check functions as $r_{e}$. Our goal when calculating Equations 2.1, 2.1', and 2.1'' is to verify if $r_{s}$ is *reasonably close* to $r_{e}$.

* *For Equations 2.1 and 2.1'':* If $n < 10^6$, we check if $r_{s}$ is within two standard deviations of the calculated value. If $n \geq 10^6$, we only compare the first and/or second digits of $r_{s}$ and $r_{e}$, as performed in Valiant (2005).
    * Since we know that our standard deviation is $\sqrt{r_{e}}$, this seems reasonable since our procedure will become less stable as $n$ decreases.
    * Checking via a range of standard deviations is not as feasible when $n \geq 10^6$, since our deviations become gradually smaller as $n$ increases.
* *For Equation 2.1':* We check if $r_{e}$ is within a range specified by the parameter $c_1$. 
    * All that is currently known about $c_1$ is the fact that $1 < c_1 < 1.1$. Since we have no other knowledge of $c_1$, we assume a generous range of $(2 - c_1) r_{e} < r_{s} < c_1 r_{e}$. This grants $r_{s}$ nearly a $10\%$ error margin when compared to $r_{e}$.

In [1]:
import numpy as np
from scipy.stats import binom

In [2]:
def B(r, p, k):
    return binom.sf(k-1, r, p)

def T(r, p, j):
    return binom.pmf(j, r, p)

In [3]:
def compare_first_x_digits(a, b, x):
    return str(a)[:x] == str(b)[:x]

# I. Two-Step, Disjoint Representation Conditions

In [4]:
def check_eq_2_1(n, p, k, r):
    expect = round(n * B(r, p, k)**2, 2)
    if n < 10**6:
        stdev = 2 * np.sqrt(expect)
        return r >= (expect - stdev) and r <= (expect + stdev)
    return compare_first_x_digits(expect, r, 2)

def check_eq_2_2(n, p, k, r):
    p_dash = B(r/2, p, k) * B(r, p, k)
    expect = B(n, p_dash, r/10)
    return round(expect, 6) == 0.0

def check_eq_2_3(n, p, k, r):
    p_dash = (1 - B(r, p, k)) * (B(r, p, k))**2
    expect = B(n, p_dash, 2*r/3)
    return round(expect, 6) == 1.0

def check_eq_2_4(n, p, k, r):
    p_dash = p * B(r, p , k)
    Y = B(n, p_dash, k)
    return round(Y, 6) == 1.0

def check_eq_2_5(n, p, k, r):
    p_dash = B(n, p * B(r/2, p, k), k)
    expect = B(r, p_dash, r/2)
    return round(expect, 6) == 0.0

def check_eq_2_6(n, p, k, r, t):
    p_dash = p * (1 - (1 - B(r, p, k))**t)
    p_ddash = B(n, p_dash * B(r, p, k), k)
    approx = B(r, p_ddash, r/2)
    return round(approx, 6) == 0.0

In [5]:
def run_twostep_disjoint_checks(n, p, k, r, t=1):
    print("For a Disjoint Representation with Two-Step Mechanisms,")
    header = ' set n={0}, d={1}, k={2}, t={3},\n and test r={4}:\n'
    print(header.format(n, round(n*p), k, t, r))
    print("Equation 2.1:", check_eq_2_1(n, p, k, r))
    print("Equation 2.2:", check_eq_2_2(n, p, k, r))
    print("Equation 2.3:", check_eq_2_3(n, p, k, r))
    print("Equation 2.4:", check_eq_2_4(n, p, k, r))
    print("Equation 2.5:", check_eq_2_5(n, p, k, r))
    print("Equation 2.6:", check_eq_2_6(n, p, k, r, t))

### Example Check for a Two-Step, Disjoint Representation

In [6]:
n = 100000
d = 512
p = d / n
k = 16

r_table_1 = 2338 # from Table 1 of Valiant (2005)

run_twostep_disjoint_checks(n, p, k, r_table_1, 1)

For a Disjoint Representation with Two-Step Mechanisms,
 set n=100000, d=512, k=16, t=1,
 and test r=2338:

Equation 2.1: True
Equation 2.2: True
Equation 2.3: True
Equation 2.4: True
Equation 2.5: True
Equation 2.6: True


## II. Two-Step, Shared Representation Conditions

In [7]:
def check_eq_2_1_dash(n, p, k, r, c_1=1.0999):
    r_dash = round(r**2 / n)
    total = 0.0
    for i in range(0, k):
        total += T(r_dash, p, i)*(B(r-r_dash, p, k-i))**2
    p_dash = B(r_dash, p, k) + total
    expect = round(n * p_dash, 6)
    dev = expect * (c_1 - 1)
    return r >= (expect - dev) and r <= (expect + dev)

def check_eq_2_2_dash(n, p, k, r):
    r_dash = round(r**2 / n)
    total = 0.0
    for i in range(0, k):
        total += T(r_dash, p, i)*(B(r-r_dash, p, k-i))*B((r/2)-r_dash, p, k-i)
    p_dash = B(r_dash, p, k) + total
    expect = B(n, p_dash, r/10)
    return round(expect, 6) == 0.0

def check_eq_2_3_dash(n, p, k, r):
    r_dash = round(r**2 / n)
    r_ddash = round(r**3 / n**2)
    r_caret = r_dash - r_ddash
    p_dash = 0.0
    for i in range(0, k+1):
        total_j = 0.0
        for j in range(0, k-i+1):
            total_l = 0.0
            for l in range(0, k-i-j+1):
                total_m = 0.0
                for m in range(0, k-i-j-l+1):
                    total_m += T(r_ddash, p, m) * (B(r-2*r_dash+r_ddash, p, k-i-j-m) * B(r-2*r_dash+r_ddash, p, k-j-l-m) * (1-B(r-2*r_dash + r_ddash, p, k-i-l-m)))
                total_l += T(r_caret, p, l) * total_m
            total_j += T(r_caret, p, j) * total_l
        p_dash += T(r_caret, p, i) * total_j
    expect = B(n, p_dash, 2*r/3)
    return round(expect, 5) == 1.0

def check_eq_2_4_dash(n, p, k, r):
    return check_eq_2_4(n, p, k, r)

def check_eq_2_5_dash(n, p, k, r):
    return check_eq_2_5(n, p, k, r)

def check_eq_2_6_dash(n, p, k, r, t=1):
    r_dash = round(r**2 / n)
    total = 0.0
    for i in range(0, k):
        total += T(r_dash, p, i)*(B(r-r_dash, p, k-i))**2
    p_dash = B(r_dash, p, k) + total
    p_ddash = p_dash * B(n, p_dash, k)
    expect = B(r, p_ddash, r/2)
    return round(expect, 6) == 0.0

In [8]:
def run_twostep_shared_checks(n, p, k, r, t=1, c_1=1.0999):
    print("For a Shared Representation with Two-Step Mechanisms,")
    header = ' set n={0}, d={1}, k={2}, t={3}, c_1={4},\n and test r={5}:\n'
    print(header.format(n, round(n*p), k, t, c_1, r))
    print("Equation 2.1':", check_eq_2_1_dash(n, p, k, r, c_1))
    print("Equation 2.2':", check_eq_2_2_dash(n, p, k, r))
    print("Equation 2.3':", check_eq_2_3_dash(n, p, k, r))
    print("Equation 2.4':", check_eq_2_4_dash(n, p, k, r))
    print("Equation 2.5':", check_eq_2_5_dash(n, p, k, r))
    print("Equation 2.6':", check_eq_2_6_dash(n, p, k, r, t))

### Example Check for a Two-Step, Shared Representation

In [9]:
n = 100000
d = 512
p = d / n
k = 16

r_table_1 = 2338 # from Table 1 of Valiant (2005)

run_twostep_shared_checks(n, p, k, r_table_1, 1, 1.0999)

For a Shared Representation with Two-Step Mechanisms,
 set n=100000, d=512, k=16, t=1, c_1=1.0999,
 and test r=2338:

Equation 2.1': True
Equation 2.2': True
Equation 2.3': True
Equation 2.4': True
Equation 2.5': True
Equation 2.6': True


## III. One-Step, Shared Representation Conditions
#### Note: An additional option for Equation 2.3'':
The calculation of Equation 2.3'' is available for a *one-step, disjoint* representation. We believe this was given due to the significant runtime complexity when calculating the shared variant. This variant can be accessed by setting the parameter $complete\_share$ of that check function to be __False__. 

Exploring this could be of interest if developing a new set of equations for a disjoint representation with one-step mechanisms, or if interested in developing any hybrid models with any partial sharing of neurons between memories.

In [10]:
def check_eq_2_1_ddash(n, p, k_m, r):
    r_dash = round(r**2 / n)
    p_dash = B((2*r) - r_dash, p, k_m)
    expect = round(n * p_dash, 2)
    if n < 10**6:
        stdev = 2 * np.sqrt(expect)
        return r >= (expect - stdev) and r <= (expect + stdev)
    return compare_first_x_digits(expect, r, 2)

def check_eq_2_2_ddash(n, p, k_m, r):
    p_dash = B(3*r/2, p, k_m) 
    expect = B(n, p_dash, r/10)
    return round(expect, 6) == 0.0

def eq_2_3_ddash_p_dash_disjoint(n, p, k_m, r):
    p_dash = 0.0
    for s in range(0, k_m):
        p_dash += T(r, p, s) * (B(r, p, k_m - s)) * (1 - B(r, p, k_m - s))
    return p_dash

def eq_2_3_ddash_p_dash_shared(n, p, k_m, r):
    r_dash = round(r**2 / n)
    r_ddash = round(r**3 / n**2)
    r_caret = r_dash - r_ddash
    r_sharp = r + 2*r_dash + r_ddash # double minus in paper?
    p_dash = 0.0
    for s in range(0, k_m):
        total_i = 0.0
        for i in range(0, k_m-s):
            total_j = 0.0
            for j in range(0, k_m-i-s):
                total_m = 0.0
                for m in range(0, k_m-i-j-s):
                    total_l = 0.0
                    for l in range(0, k_m-i-j-m-s):
                        total_l += T(r_caret, p, l) * ((B(r_sharp, p, k_m-s-i-j-l-m)) * (1 - B(r_sharp, p, k_m-s-i-j-l-m)))
                    total_m += T(r_ddash, p, m) * total_l
                total_j += T(r_caret, p, j) * total_m
            total_i += T(r_caret, p, i) * total_j
        p_dash += T(r_sharp, p, s) * total_i
    return p_dash

def check_eq_2_3_ddash(n, p, k_m, r, complete_share=True):
    if not complete_share:
        p_dash = eq_2_3_ddash_p_dash_disjoint(n, p, k_m, r)
    else:
        p_dash = eq_2_3_ddash_p_dash_shared(n, p, k_m, r)
    expect = B(n, p_dash, 2*r/3)
    return round(expect, 6) == 1.0

def check_eq_2_4_ddash(n, p, k_a, r):
    p_dash = p * B(r, p, k_a)
    Y = B(n, p_dash, k_a)
    return round(Y, 6) == 1.0

def check_eq_2_5_ddash(n, p, k_a, r):
    p_dash = p * B(r/2, p, k_a)
    p_ddash = B(n, p_dash, k_a)
    expect = B(r, p_ddash, r/2)
    return round(expect, 6) == 0.0

def check_eq_2_6_ddash(n, p, k_a, r):
    r_dash = round(r**2 / n)
    total = 0.0
    for i in range(0, k_a):
        total += T(r_dash, p, i)*(B(r-r_dash, p, k_a-i))**2
    p_dash = p * (B(r_dash, p, k_a) + total)
    p_ddash = B(n, p_dash, k_a)
    expect = B(r, p_ddash, r/2)
    return round(expect, 6) == 0.0

In [11]:
def run_onestep_shared_checks(n, p, k, k_adj, r, complete_share=True):
    k_a = k
    k_m = round(k_adj * k_a)
    if not complete_share:
        print("For a Partially-Shared Representation with One-Step Mechanisms,")
    else:
        print("For a Shared Representation with One-Step Mechanisms,")
    header = ' set n={0}, d={1}, k_a={2}, k_m={3},\n and test r={4}:\n'
    print(header.format(n, round(n*p), k_a, k_m, r))
    print("Equation 2.1'':", check_eq_2_1_ddash(n, p, k_m, r))
    print("Equation 2.2'':", check_eq_2_2_ddash(n, p, k_m, r))
    print("* Calculations of Equation 2.3'' (Shared) take a while... *", end='\r')
    eq_2_3_ddash_result = check_eq_2_3_ddash(n, p, k_m, r, complete_share)
    if not complete_share:
        print("Equation 2.3'' (Disjoint):", eq_2_3_ddash_result, ' '*45)
    else:
        print("Equation 2.3'':", eq_2_3_ddash_result, ' '*45)
    print("Equation 2.4'':", check_eq_2_4_ddash(n, p, k_a, r))
    print("Equation 2.5'':", check_eq_2_5_ddash(n, p, k_a, r))
    print("Equation 2.6'':", check_eq_2_6_ddash(n, p, k_a, r))

### Example Check for a One-Step, Shared Representation

In [12]:
n = 100000
d = 512
p = d / n
k = 16

k_adj = 2.0
r_table_3 = 2134 # from Table 3 of Valiant (2005)

run_onestep_shared_checks(n, p, k, k_adj, r_table_3, complete_share=True)

For a Shared Representation with One-Step Mechanisms,
 set n=100000, d=512, k_a=16, k_m=32,
 and test r=2134:

Equation 2.1'': True
Equation 2.2'': True
Equation 2.3'': True                                              
Equation 2.4'': True
Equation 2.5'': True
Equation 2.6'': True


## Example Tests for Finding New Solutions of $r$

#### Test for a "worm-sized" shared representation that meets some conditions:

In [13]:
n = 500
d = 128
p = d / n
k = 16
k_adj = 1.5

r_test = 37

run_onestep_shared_checks(n, p, k, k_adj, r_test, complete_share=True)

For a Shared Representation with One-Step Mechanisms,
 set n=500, d=128, k_a=16, k_m=24,
 and test r=37:

Equation 2.1'': True
Equation 2.2'': False
Equation 2.3'': True                                              
Equation 2.4'': False
Equation 2.5'': True
Equation 2.6'': True


#### Test for a "human-sized," *partially-shared* representation that meets some conditions:

In [14]:
n = 100000000000
d = 8192
p = d / n
k = 64
k_adj = 2.0

r_test = 512000000

run_onestep_shared_checks(n, p, k, k_adj, r_test, complete_share=False)

For a Partially-Shared Representation with One-Step Mechanisms,
 set n=100000000000, d=8192, k_a=64, k_m=128,
 and test r=512000000:

Equation 2.1'': False
Equation 2.2'': True
Equation 2.3'' (Disjoint): False                                              
Equation 2.4'': False
Equation 2.5'': True
Equation 2.6'': True


## Potential Future Work
* Verify the bounds given in Chapter 5 of the paper
* Explore the $c_2$ variable from Chapter 7 of the paper
* Incorporate all calculations from the paper and related work
* Add a simple method to automatically search for new solutions
    * This was done using binary search in the paper